In [1]:
import barmuscomp.scripts.default_path as paths
import barmuscomp.scripts.overall_scripts as scr
import as_seg.data_manipulation as dm
import as_seg.CBM_algorithm as CBM
import as_seg.autosimilarity_computation as as_comp
#import as_seg.barwise_input as bi
from barmuscomp.model.current_plot import *

import math
import numpy as np
import pandas as pd
import mirdata
import os
import tensorly as tl

In [2]:
colors = ['tab:green', 'tab:purple','tab:red','tab:olive']
markers = ['^', 's', 'o','v']

convolution_type = "14_bands"
autosimilarity_types = ["Cosine", "Covariance", "RBF"]
pen_function = "modulo8"
features = ["pcp"]#"nn_log_mel_grill", "mel_grill", 
features_labels = ["Chromagram"]#"NNLMS", "Mel", 

dimensions_range = [12,16,20,24,28,32,36,40,44,48]
lambda_penalty = 1 #np.concatenate([np.arange(0.1, 2.1, 0.1)])

In [3]:
def compute_this_penalty_lambda_fitting(penalty_func, lambda_penalty, dimensions_range, autosimilarity_type = "cosine", feature = "nn_log_mel_grill", convolution_type = "14_bands", update_rule = "hals", beta = None):
    idx_song_max = 101 # Should be 101 for all the dataset, other values should be for tests only
    songs_range = range(1,idx_song_max) 
    subdivision = 96
    hop_length = 32
    hop_length_seconds = hop_length/44100
    if feature == "pcp":
        dim_range_W = [12]
        init = "chromas"
    else:
        dim_range_W = dimensions_range
        init = "tucker"
    
    results_songs_lambdas = math.inf * np.ones((len(songs_range), len(dim_range_W), len(dimensions_range), len(dimensions_range), 2, 3)) # Songs, autosimilarity types, tol, metrics
    repartition_songs_lambdas = np.zeros((len(dim_range_W), len(dimensions_range), len(dimensions_range), 40)) # Songs, autosimilarity types, tol, metrics

    for idx_song, song_name in enumerate(songs_range):
        spectrogram, bars, references_segments = scr.load_spec_annot_song_RWC(song_name, feature, hop_length)
        for idx_W, dim_W in enumerate(dim_range_W):
            for idx_H, dim_H in enumerate(dimensions_range):
                for idx_Q, dim_Q in enumerate(dimensions_range):
                    if update_rule == "hals":
                        path_for_ntd = f"{paths.path_data_persisted_rwc}/ntd/{dim_W}_{dim_H}_{dim_Q}"
                        persisted_arguments = f"_{song_name}_{feature}_{init}_96"
                    elif update_rule == "mu":
                        path_for_ntd = f"{paths.path_data_persisted_rwc}/ntd_mu/{dim_W}_{dim_H}_{dim_Q}"
                        persisted_arguments = f"mu_slow_{song_name}_beta{beta}_{feature}_{init}_96_n_iter_max1000"
                    a_factor_path = f"{path_for_ntd}/factors{persisted_arguments}.npy"
                    Q_matrix = np.load(a_factor_path, allow_pickle=True)[2]
                    autosimilarity_Q_matrix = as_comp.switch_autosimilarity(Q_matrix, similarity_type = autosimilarity_type, normalise = True)
                    segments = CBM.compute_cbm(autosimilarity_Q_matrix, penalty_weight = lambda_penalty, penalty_func = penalty_func, convolution_type = convolution_type)[0]                
                    segments_in_time = dm.segments_from_bar_to_time(segments, bars)
                    for end, start in segments:
                        repartition_songs_lambdas[idx_W, idx_H, idx_Q, start - end] += 1
                    prec05, rap05, f_mes05 = dm.compute_score_of_segmentation(references_segments, segments_in_time, window_length = 0.5)
                    prec3, rap3, f_mes3 = dm.compute_score_of_segmentation(references_segments, segments_in_time, window_length = 3)
                    this_score = [[round(prec05,4),round(rap05,4),round(f_mes05,4)], [round(prec3,4),round(rap3,4),round(f_mes3,4)]]
                    results_songs_lambdas[idx_song, idx_W, idx_H, idx_Q] = this_score
    
    best_prec_05 = 0
    best_prec_3 = 0
    best_rec_05 = 0
    best_rec_3 = 0
    best_fmes_05 = 0
    best_fmes_3 = 0
    for idx_sog, song_name in enumerate(songs_range):
        arg_max = np.argmax(results_songs_lambdas[idx_sog, :,:,:,0, 2] + results_songs_lambdas[idx_sog, :,:,:,1, 2])
        best_args = np.unravel_index(arg_max, np.array(results_songs_lambdas[idx_sog, :,:,:,0, 2]).shape)
        best_prec_05 += results_songs_lambdas[idx_sog, best_args[0], best_args[1], best_args[2],0, 0]
        best_prec_3 += results_songs_lambdas[idx_sog, best_args[0], best_args[1], best_args[2],1, 0]
        best_rec_05 += results_songs_lambdas[idx_sog, best_args[0], best_args[1], best_args[2],0, 1]
        best_rec_3 += results_songs_lambdas[idx_sog, best_args[0], best_args[1], best_args[2],1, 1]
        best_fmes_05 += results_songs_lambdas[idx_sog, best_args[0], best_args[1], best_args[2],0, 2]
        best_fmes_3 += results_songs_lambdas[idx_sog, best_args[0], best_args[1], best_args[2],1, 2]

    return best_fmes_05/len(songs_range), best_fmes_3/len(songs_range)


In [4]:
def plot_kernels_as(results_diff_as, kernels_or_features, autosimilarities, tol, title = None):
    # Order results_diff_as: kernels ou features, as, tol
    subaxis = autosimilarities
    plt.rcParams.update({'font.size': 14})
    
    width=0.25
    gap = (len(autosimilarities) + 1)*width
    colors = ["#62d5b4", "#317373", "#184a4a"]#['skyblue', 'steelblue', 'darkorange', 'brown']
    
    fig, ax = plt.subplots(1, 1, figsize=(11,4))
    ax.set_ylim( 25, 105 )
    tab = []
    current_start = 0
    for i in range(len(autosimilarities)):
        indexes = [current_start + j*gap for j in range(len(kernels_or_features))]
        tab.append(ax.bar(indexes, results_diff_as[:,i,tol], color=colors[i], width=width, align='edge'))
        for idx_x, x in enumerate(indexes):
            height = results_diff_as[idx_x, i, tol] 
            plt.text(x +width/2,height + 2, f"{height}%", ha='center', va='bottom', rotation = 90)
        current_start += width

    ax.set_xticks([current_start/2 + j*gap for j in range(len(kernels_or_features))])
    ax.set_xticklabels(kernels_or_features)
    if tol == 1:
        ax.legend(labels=subaxis, loc='lower left')
    else:
        ax.legend(labels=subaxis, loc='lower left')

    ax.set_ylabel("F measure (percentage)")
    #fig.savefig("C:/Users/amarmore/Desktop/Mes papiers/figures/results_barcomp_05.pdf", bbox_inches = "tight")#, format='png')
    if title != None:
        plt.title(title)
    plt.show()

In [5]:
update_rule = "hals"
beta = None
print(f"Update: {update_rule}, beta: {beta}")

print(f"Kernel: {convolution_type}")

for idx_feat, feature in enumerate(features):
    print(f"Feature: {feature}")
    indexes = []
    results_this_function = math.inf * np.ones((len(autosimilarity_types), 2, 3))
    for idx_as, autosimil_type in enumerate(autosimilarity_types):
        print(f"Autosimilarity type: {autosimil_type}")
        f_05, f_3 = compute_this_penalty_lambda_fitting(pen_function, lambda_penalty, dimensions_range, autosimilarity_type = autosimil_type, feature = feature, convolution_type = convolution_type, update_rule = update_rule, beta = beta)
        print(f"Oracle: F05 {f_05}, F3 {f_3}")

Update: hals, beta: None
Kernel: 14_bands
Feature: pcp
Autosimilarity type: Cosine
Oracle: F05 0.742499, F3 0.8525010000000001
Autosimilarity type: Covariance
Oracle: F05 0.7176540000000001, F3 0.7960490000000001
Autosimilarity type: RBF
Oracle: F05 0.7391250000000003, F3 0.8449819999999997
